# Práctica 1. Resolución de problemas con búsqueda 
## Sistemas Inteligentes
### Belén Díaz Agudo, Ismael Sagredo Olivenza

La práctica está organizada en 2 partes obligatorias y una opcional:
* Parte I: se muestra a través de ejemplos resueltos cómo se representan algunos problemas clásicos como el de las jarras, el problema de los misioneros o el problema del ocho puzzle. 
* Parte II: se muestra el uso de los algoritmos de búsqueda exhaustiva (ciega y heurística) vistos en clase. 

Opcional
* Ejercicio 6 del notebook, realizar la parte II con el 16 Puzle.

En el notebook encontraras claramente identificados los lugares en los que debes incluir código o comentarios.  
Cuando termines los ejercicios entrega **este archivo en el campus**.  
Debes incluir al comienzo una celda de markdown con el nombre completo de los miembros del grupo y número de grupo.

Los comentarios razonados de los ejercicios son la parte más importante de esta práctica. 

## Parte I: Representación de problemas de espacios de estados.

Como hemos visto en clase la representación de un problema de espacio de estados consiste en:
* Representar estados y acciones mediante una estructura de datos.
* Definir: estado_inicial, es_estado_final(_), acciones(_), aplica(_,_) y
  coste_de_aplicar_accion, si el problema tiene coste.

 La siguiente clase Problem representa este esquema general de cualquier
 problema de espacio de estados. Un problema concreto será una subclase de
 Problema, y requerirá implementar acciones, aplica y eventualmente __init__, actions,
 goal_test. La función coste_de_aplicar_accion la hemos incluido nosotros
 
El primer paso es importar el código que necesitamos de search.py de AIMA y usar la clase Problem. 

In [23]:
from search import Problem

In [24]:
# Para que funcione el import el archivo search.py tiene que estar en la mimsa carpeta que este notebook.
# Se puede copiar el archivo search.py a la mimsa carpeta que este notebook.
# Otra opcion para poner un prefijo a la ruta antes de importar es:
import sys
sys.path.append('aima/')
import search

In [25]:
## Copiamos aquí la definición de la clase Problem para facilitar la explicación.

class Problem(object):

    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2.  If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value.  Hill-climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError

    def coste_de_aplicar_accion(self, estado, accion):
        """Hemos incluido está función que devuelve el coste de un único operador (aplicar accion a estado). Por defecto, este
        coste es 1. Reimplementar si el problema define otro coste """ 
        return 1

Ahora vamos a ver un ejemplo de cómo definir un problema como subclase
de problema. En concreto, el problema de las jarras, visto en clase que es muy sencillo. 

In [26]:
class Jarras(Problem):
    """Problema de las jarras:
    Representaremos los estados como tuplas (x,y) de dos números enteros,
    donde x es el número de litros de la jarra de 4 e y es el número de litros
    de la jarra de 3"""

    def __init__(self):
        self.initial = (0,0)

    def actions(self,estado):
        jarra_de_4=estado[0]
        jarra_de_3=estado[1]
        accs=list()
        if jarra_de_4 > 0:
            accs.append("vaciar jarra de 4")
            if jarra_de_3 < 3:
                accs.append("trasvasar de jarra de 4 a jarra de 3")
        if jarra_de_4 < 4:
            accs.append("llenar jarra de 4")
            if jarra_de_3 > 0:
                accs.append("trasvasar de jarra de 3 a jarra de 4")
        if jarra_de_3 > 0:
            accs.append("vaciar jarra de 3")
        if jarra_de_3 < 3:
            accs.append("llenar jarra de 3")
        return accs

    def result(self,estado,accion):
        j4=estado[0]
        j3=estado[1]
        if accion=="llenar jarra de 4":
            return (4,j3)
        elif accion=="llenar jarra de 3":
            return (j4,3)
        elif accion=="vaciar jarra de 4":
            return (0,j3)
        elif accion=="vaciar jarra de 3":
            return (j4,0)
        elif accion=="trasvasar de jarra de 4 a jarra de 3":
            return (j4-3+j3,3) if j3+j4 >= 3 else (0,j3+j4)
        else: #  "trasvasar de jarra de 3 a jarra de 4"
            return (j3+j4,0) if j3+j4 <= 4 else (4,j3-4+j4)

    def goal_test(self,estado):
        return estado[0]==2


Vamos a probar algunos ejemplos.

In [27]:
p =Jarras()
p.initial

(0, 0)

In [28]:
p.actions(p.initial)

['llenar jarra de 4', 'llenar jarra de 3']

In [29]:
p.result(p.initial,"llenar jarra de 4")

(4, 0)

In [30]:
p.coste_de_aplicar_accion(p.initial,"llenar jarra de 4")

1

In [31]:
p.goal_test(p.initial)

False

### Problema de los misioneros

In [32]:
# Creamos la clase ProblemaMisioneros con los elementos que representarán el problema. 
class ProblemaMisioneros(Problem):
    ''' Clase problema (formalizacion de nuestro problema) siguiendo la
        estructura que aima espera que tengan los problemas.'''
    def __init__(self, initial, goal=None):
        '''Inicializacion de nuestro problema.'''
        Problem.__init__(self, initial, goal)
        # cada accion tiene un texto para identificar al operador y despues una tupla de dos elementos con la
        # cantidad de misioneros y canibales que se mueven en la canoa
        self._actions = [('1c', (0, 1)), ('1m', (1, 0)), ('2c', (0, 2)), ('2m', (2, 0)), ('1m1c', (1, 1))]

    def actions(self, s):
        '''Devuelve las acciones validas para un estado.'''
        # las acciones validas para un estado son aquellas que al aplicarse
        # nos dejan en otro estado valido
        return [a for a in self._actions if self._is_valid(self.result(s, a))]

    def _is_valid(self, s):
        '''Determina si un estado es valido o no.'''
        # un estado es valido si no hay mas canibales que misioneros en ninguna
        # orilla, y si las cantidades estan entre 0 y 3
        return (s[0] >= s[1] or s[0] == 0) and ((3 - s[0]) >= (3 - s[1]) or s[0] == 3) and (0 <= s[0] <= 3) and (0 <= s[1] <= 3)

    def result(self, s, a):
        '''Devuelve el estado resultante de aplicar una accion a un estado
           determinado.'''
        # el estado resultante tiene la canoa en el lado opuesto, y con las
        # cantidades de misioneros y canibales actualizadas segun la cantidad
        # que viajaron en la canoa
        if s[2] == 0:
            return (s[0] - a[1][0], s[1] - a[1][1], 1)
        else:
            return (s[0] + a[1][0], s[1] + a[1][1], 0)



In [33]:
# creamos un problema a partir de nuestra formalizacion de ProblemaMisioneros
# como parametros le pasamos el estado inicial, y el estado objetivo que esperamos
misioneros = ProblemaMisioneros((3, 3, 0), (0, 0, 1))

# Asegurate de que entiendes la formalización del problema y haz algunas pruebas con la representación del problema de los misioneros. 
# En la siguiente parte vamos a usar las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura la llamada sería:  breadth_first_tree_search(estado).solution()

In [34]:
misioneros.initial

(3, 3, 0)

In [35]:
misioneros.actions(misioneros.initial)

[('1c', (0, 1)), ('2c', (0, 2)), ('1m1c', (1, 1))]

### Representación del problema del puzzle de 8

Vamos a definir la clase Ocho_Puzzle, que implementa la representación del problema del 8-puzzle visto en clase. 
Se os proporciona una versión incompleta y tendréis que completar el código que se presenta a continuación, en los lugares marcados con interrogantes.

### 8 Puzzle 

Tablero 3x3 cuyo objetivo es mover la configuración de las piezas desde un estado inicial dado a un estado objetivo moviendo las fichas al espacio en blanco. 

ejemplo:- 

                  Inicial                             Goal 
              | 7 | 2 | 4 |                       | 1 | 2 | 3 |
              | 5 | 0 | 6 |                       | 4 | 5 | 6 |
              | 8 | 3 | 1 |                       | 7 | 8 | 0 |
              
Hay 9! configuraciones iniciales pero ojo! porque no todas tienen solución. **Tenlo en cuenta al hacer las pruebas**. 

### EJERCICIO 1. Completa la definición de los operadores en el problema del Puzle de 8. 

In [36]:
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if pos_hueco not in (6, 7, 8):
            accs.append("Mover hueco abajo")
        
        if pos_hueco not in (0, 3, 6):
            accs.append("Mover hueco izquierda")

        if pos_hueco not in (2, 5, 8):
            accs.append("Mover hueco derecha")

        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        
       ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+3]
            l[pos_hueco+3] = 0
        
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0

        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0

        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """""
        return 1
    
    def check_solvability(self, state):
        inversion = 0
        for i in range(len(state)):
            for j in range(i + 1, len(state)):
                if state[i] > state[j] and state[i] != 0 and state[j] != 0:
                    inversion += 1
        return inversion % 2 == 0

#### Una vez completada la definición de la clase podrás probar los siguientes ejemplos.

In [37]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [38]:
p8.actions(p8.initial)
#Respuesta: ['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [39]:
p8.result(p8.initial,"Mover hueco arriba")

(2, 8, 3, 1, 0, 4, 7, 6, 5)

In [40]:
p8.result(p8.initial,"Mover hueco abajo")

IndexError: list index out of range

In [ ]:
p8.result(p8.initial,"Mover hueco derecha")

(2, 8, 3, 1, 6, 4, 7, 5, 0)

In [ ]:
p8.coste_de_aplicar_accion(p8.initial,"Mover hueco abajo")

1

### EJERCICIO 2.  Usando la representación anterior representa el problema del puzle de 16 piezas.

In [ ]:
class Quince_Puzzle(Problem): 
    ## Completa la definición del problema
    """Problema del 16-puzzle.  Los estados serán tuplas de 16 elementos,
    permutaciones de los números del 0 al 15 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
    
    def actions(self,estado):
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0, 1, 2, 3):
            accs.append("Mover hueco arriba")
        
        ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if pos_hueco not in (12, 13, 14, 15):
            accs.append("Mover hueco abajo")
        
        if pos_hueco not in (0, 4, 8, 12):
            accs.append("Mover hueco izquierda")

        if pos_hueco not in (3, 7, 11, 15):
            accs.append("Mover hueco derecha")

        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-4]
            l[pos_hueco-4] = 0
        
       ### COMPLETA LA DEFINICIÓN DE LOS OPERADORES. 
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco+4]
            l[pos_hueco+4] = 0
        
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0

        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """""
        return 1

Realiza algunas pruebas para comprobar que la clase está bien definida. 

In [ ]:
## Completa con las pruebas en este espacio

p15 = Quince_Puzzle((2, 15, 3, 1, 6, 4, 7, 12, 5, 11, 10, 9, 13, 8, 0, 14))
p15.initial

(2, 15, 3, 1, 6, 4, 7, 12, 5, 11, 10, 9, 13, 8, 0, 14)

In [ ]:
p15.actions(p15.initial)

['Mover hueco arriba', 'Mover hueco izquierda', 'Mover hueco derecha']

In [ ]:
p15.result(p15.initial,"Mover hueco arriba")

(2, 15, 3, 1, 6, 4, 7, 12, 5, 11, 0, 9, 13, 8, 10, 14)

In [ ]:
p15.result(p15.initial,"Mover hueco abajo")

IndexError: list index out of range

In [ ]:
p15.result(p15.initial,"Mover hueco izquierda")

(2, 15, 3, 1, 6, 4, 7, 12, 5, 11, 10, 9, 13, 0, 8, 14)

In [ ]:
p15.result(p15.initial,"Mover hueco derecha")

(2, 15, 3, 1, 6, 4, 7, 12, 5, 11, 10, 9, 13, 8, 14, 0)

In [ ]:
p15.coste_de_aplicar_accion(p15.initial, "Mover hueco abajo")

1

## Parte II: Experimentación con los algoritmos implementados. Ejecución de los algoritmos de búsqueda de soluciones para una instancia del Problema.

En primer lugar vamos a ver cómo funciona la búsqueda ciega (en anchura y en profundidad) para encontrar soluciones para los tres problemas anteriores: el problema de las jarras, los misioneros y el problema del ocho puzzle con distintos estados iniciales.

In [ ]:
# Cargamos el módulo con los algoritmos de búsqueda.
from search import *
from search import breadth_first_tree_search, depth_first_tree_search, depth_first_graph_search, breadth_first_graph_search

Para que la importación de search funcione el archivo search.py y utils.py tienen que estar en la misma carpeta que este notebook o poner la ruta correspondiente.

In [ ]:
# una opcion para poner un prefijo a la ruta antes de importar es:
import sys
sys.path.append('aima-python/')
import search

In [ ]:
## A continuación se comprueba la resolución del problema de las jarras con el método de búsqueda en anchura.

In [ ]:
breadth_first_tree_search(Jarras()).solution()

['llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3',
 'vaciar jarra de 3',
 'trasvasar de jarra de 4 a jarra de 3',
 'llenar jarra de 4',
 'trasvasar de jarra de 4 a jarra de 3']

In [ ]:
depth_first_graph_search(Jarras()).solution()

['llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'llenar jarra de 3',
 'trasvasar de jarra de 3 a jarra de 4',
 'vaciar jarra de 4',
 'trasvasar de jarra de 3 a jarra de 4']

### EJERCICIO 3. Prueba los algoritmos de búsqueda ciega con el problema de los misioneros y con el  puzzle de 8 y de 16.  Comenta al final cómo se comportan los algoritmos en cuanto a eficiencia y resultados. 

In [ ]:
# Usaremos las implementaciones de los algoritmos de búsqueda de AIMA para 
# resolver los problemas que hemos representado. 

#Por ejemplo, para resolver el problema de los misioneros con 
# el método de busqueda en anchura 

breadth_first_tree_search(misioneros).solution()

# Realiza pruebas con los diferentes algoritmos observando el comportamiento.

[('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2m', (2, 0)),
 ('1m1c', (1, 1)),
 ('2m', (2, 0)),
 ('1c', (0, 1)),
 ('2c', (0, 2)),
 ('1c', (0, 1)),
 ('2c', (0, 2))]

In [ ]:
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
p8.initial

(2, 8, 3, 1, 6, 4, 7, 0, 5)

In [ ]:
p8.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [ ]:
breadth_first_tree_search(Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))).solution()
## La llamada corresponde al algoritmo de busqueda en anchura sin control de repetidos.  
## Esta llamada no funciona, se queda en [*]
## Si es método de busqueda en anchura es completo.. ¿no debería terminar? ¿qué crees que está pasando?
## ...............................................................................

# Se necesitaría implementar un control de repetidos, ya que el problema contiene estados repetidos
# por lo que el algoritmo se queda en un bucle infinito, visitando los mismos estados una y otra vez
# sin poder avanzar

KeyboardInterrupt: 

In [ ]:
# Comprueba si el estado inicial (2, 8, 3, 1, 6, 4, 7, 0, 5) tiene solución.
p8 = Ocho_Puzzle((2, 8, 3, 1, 6, 4, 7, 0, 5))
if p8.check_solvability(p8.initial):
    sol= breadth_first_tree_search(p8).solution()
else: 
    sol="Este estado inicial no tiene solucion"

In [ ]:
sol

'Este estado inicial no tiene solucion'

In [ ]:
breadth_first_tree_search(Ocho_Puzzle((1,2,3,4,5,6,0,7,8))).solution()

['Mover hueco derecha', 'Mover hueco derecha']

In [ ]:
estado = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))

In [ ]:
breadth_first_tree_search(estado).solution()
# Respuesta: ['arriba', 'izquierda', 'arriba', 'izquierda', 'abajo', 'derecha', 'derecha', 'abajo']

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [ ]:
depth_first_graph_search(estado).solution()

NameError: name 'depth_first_graph_search' is not defined

In [ ]:
breadth_first_graph_search(estado).solution()

['Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco arriba',
 'Mover hueco izquierda',
 'Mover hueco abajo',
 'Mover hueco derecha',
 'Mover hueco derecha',
 'Mover hueco abajo']

In [ ]:
breadth_first_tree_search(Quince_Puzzle((1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 0, 15))).solution() 

['Mover hueco derecha']

In [ ]:
estado = Quince_Puzzle((2, 15, 3, 1, 6, 4, 7, 12, 5, 11, 10, 9, 13, 8, 0, 14))

depth_first_graph_search(estado).solution()

KeyboardInterrupt: 

#### En este ejercicio se ha podido observar los resultados y tiempo de la ejecución de los algoritmos de búsqueda ciega.  Escribe aquí tus conclusiones respecto a la resolución de los problemas con búsqueda ciega:
- Misioneros: 
    * La búsqueda en anchura encuentra una solución óptima si existe
    * La búsqueda en profundidad puede no encontrar una solución óptima, pero puede ser más eficiente
- Puzle de 8:
    * Ambos pueden encontrar una solución si existe, pero en anchura garantiza la solución óptima
- Puzle de 16: 
    * Al tener muchos más estados, hace que ninguno de los algoritmos de búsqueda ciega no sean demasiado útiles

**********************************************************************************
Puedes añadir todo el espacio que necesites.

### EJERCICIO 4:  Definición de heurísticas

In [ ]:
#### 4.1. Para el problema de los misioneros define una heurística y comenta sus propiedades
# Puedes utilizar alguna de las heurísticas vistas en clase. 

# Heurísticas para el problema de los misioneros.
import math

def linear(node):
    state = node.state 
    misioneros_izquierda, canibales_izquierda, _ = state
    distancia = misioneros_izquierda + canibales_izquierda
    
    return distancia


def manhattan(node):
    state = node.state
   
    misioneros_izquierda, canibales_izquierda, _ = state

    goal = (0, 0, 0)

    mhd = abs(misioneros_izquierda - goal[0]) + abs(canibales_izquierda - goal[1])
    
    return mhd
                
def sqrt_manhattan(node):
    state = node.state
    
    misioneros_izquierda, canibales_izquierda, _ = state

    goal = (0, 0, 0)
    
    mhd = abs(misioneros_izquierda - goal[0]) + abs(canibales_izquierda - goal[1])

    return math.sqrt(mhd)

def max_heuristic(node):
    score1 = manhattan(node)
    score2 = linear(node)
    return max(score1, score2)


### EJERCICIO 5. 
#### Usar las implementaciones de AIMA de los algoritmos de búsqueda_coste_uniforme, busqueda_primero_el_mejor y búsqueda_a_estrella (con las heurísticas anteriores) para resolver el problema de los misioneros y algun ejemplo del puzle de 8 y puzle de 16. 

#### Comparar los costes temporales usando %timeit y comentar los resultados.

In [41]:
####  Vamos a probar el 8-puzzle utilizando el siguiente **estado inicial** 

#              +---+---+---+
#              | 2 | 4 | 3 |
#              +---+---+---+
#              | 1 | 5 | 6 |
#              +---+---+---+
#              | 7 | 8 | H |
#              +---+---+---+

puzle = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0)) 
astar_search(puzle).solution()

NameError: name 'astar_search' is not defined

In [ ]:
astar_search(puzle,manhattan).solution()

NameError: name 'astar_search' is not defined

In [ ]:
puzle.initial

(2, 4, 3, 1, 5, 6, 7, 8, 0)

In [ ]:
puzle.goal

(1, 2, 3, 4, 5, 6, 7, 8, 0)

In [ ]:
astar_search(puzle,linear).solution()

NameError: name 'memoize' is not defined

In [ ]:
astar_search(puzle,max_heuristic).solution()

NameError: name 'astar_search' is not defined

¿Has notado diferencias en los tiempos de ejecución? ¿y en los resultados?
Vamos a medirlo 

In [ ]:
puzzle_1 = Ocho_Puzzle((2, 4, 3, 1, 5, 6, 7, 8, 0))
puzzle_2 = Ocho_Puzzle((1, 2, 3, 4, 5, 6, 0, 7, 8))
puzzle_3 = Ocho_Puzzle((1, 2, 3, 4, 5, 7, 8, 6, 0))

In [ ]:
%%timeit
astar_search(puzzle_1, linear)
astar_search(puzzle_2, linear)
astar_search(puzzle_3, linear)

NameError: name 'astar_search' is not defined

In [ ]:
%%timeit
astar_search(puzzle_1, manhattan)
astar_search(puzzle_2, manhattan)
astar_search(puzzle_3, manhattan)

In [ ]:
%%timeit
astar_search(puzzle_1, sqrt_manhattan)
astar_search(puzzle_2, sqrt_manhattan)
astar_search(puzzle_3, sqrt_manhattan)

In [ ]:
%%timeit
astar_search(puzzle_1, max_heuristic)
astar_search(puzzle_2, max_heuristic)
astar_search(puzzle_3, max_heuristic)

## Parte opcional

## Ejercicio 6

####  Realiza pruebas con otros algoritmos o con algún problema que definais vosotros.  

Algunas pistas, podeis usar bidirectional_search y comparar con la busqueda no informada normal. recursive_best_first_search, hill_climbing, simulated_annealing, etc.


#### Justifica aquí lo que has hecho y que conclusiones has sacado.

----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
----------------------------------------------------------------------------
